# Обучение моделей в Pytorch

In [ ]:
import os
import random
from os.path import join as pjoin
from shutil import rmtree

import albumentations
import numpy as np
import torch
from PIL import Image
from accelerate import Accelerator
from albumentations.pytorch.transforms import ToTensorV2
from matplotlib import pyplot as plt
from tqdm import tqdm
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets

from checkpointer import CheckpointSaver, load_checkpoint
from dataset import CLASS_WEIGHTS, CustomVOCSegmentation, convert_label_to_color
from deeplabv3plus.modeling import deeplabv3plus_resnet50
from loss import CrossEntropyDiceLoss
from metric import MeanIoU
from train import count_pytorch_model_params, train
from unet import UNet
from unet_custom import CustomUNet
from utils import seed_everything

In [ ]:
seed_everything(42, torch_deterministic=False)

## Аугментации

Трансформации/аугментации для исходных изображений и масок/таргетов.

Аугментации для задач компьютерного хрения: https://albumentations.ai/

In [ ]:
IMAGE_SIZE = 256
MEAN = np.array([0.485, 0.456, 0.406])
STD = np.array([0.229, 0.224, 0.225])

train_transforms = albumentations.Compose(
    [
        albumentations.SmallestMaxSize(max_size=IMAGE_SIZE),
        albumentations.CropNonEmptyMaskIfExists(height=IMAGE_SIZE, width=IMAGE_SIZE),
        # albumentations.Resize(IMAGE_SIZE, IMAGE_SIZE),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.ShiftScaleRotate(
            shift_limit=0.05,
            scale_limit=0.1,
            rotate_limit=15,
            p=0.5,
        ),
        albumentations.RandomCrop(IMAGE_SIZE, IMAGE_SIZE, p=1.0),
        albumentations.OneOf(
            [
                albumentations.CLAHE(p=0.3),
                albumentations.RandomBrightnessContrast(p=0.5),
                albumentations.HueSaturationValue(p=0.3),
            ],
            p=0.8,
        ),
        albumentations.Normalize(mean=MEAN, std=STD),
        ToTensorV2(),
    ]
)

val_transforms = albumentations.Compose(
    [
        albumentations.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE, p=1.0),
        albumentations.Normalize(mean=MEAN, std=STD),
        ToTensorV2(),
    ]
)

## Dataset

Набор данных Pascal VOC. Рассмотрим его версию для задачи сегментации.

Сайт: http://host.robots.ox.ac.uk/pascal/VOC/

Лидерборд за 2012 год: http://host.robots.ox.ac.uk:8080/leaderboard/displaylb_main.php?challengeid=11&compid=5

При тех или иных проблемах со скачиванием с сайта соревнования, скачайте и распакуйте архив в папку `data` (`data/VOCdevkit`) отсюда: https://disk.yandex.ru/d/1jS3yBBN7YdZ-w

In [ ]:
train_dataset = CustomVOCSegmentation(
    root="../data",
    year="2012",
    image_set="train",
    download=False,  # True
    transform=train_transforms,  # transform, not transforms!
)

val_dataset = CustomVOCSegmentation(
    root="../data",
    year="2012",
    image_set="val",
    download=False,
    transform=val_transforms,
)

In [ ]:
image, target = train_dataset[0]

image_orig = image * STD[:, None, None] + MEAN[:, None, None]

fig, ax = plt.subplots(1, 3, figsize=(9, 18))
ax[0].imshow(image_orig.numpy().transpose(1, 2, 0) + 1e-5)
ax[1].imshow(image.numpy().transpose(1, 2, 0))
ax[2].imshow(convert_label_to_color(target.numpy()))

## UNet model

Статья: https://arxiv.org/abs/1505.04597

In [ ]:
model = UNet(in_channels=3, out_channels=21)
print(model)

In [ ]:
count_pytorch_model_params(model)

## Accelerator

"Accelerate — это библиотека, которая позволяет запускать один и тот же код PyTorch в любой распределенной конфигурации, добавляя всего четыре строки кода! Короче говоря, обучение и вывод в больших масштабах стали простыми, эффективными и адаптируемыми". (c)

Сайт: https://huggingface.co/docs/accelerate/index

In [ ]:
accelerator = Accelerator(cpu=False, mixed_precision="no")

## Checkpointer

Класс для сохранения наилучших версий модели в процессе обучения.

См. класс `Checkpointer` в `train.py`

## Обучаем модель

См. `train.py`

In [ ]:
CLASSES_NUM = 21

BACKBONE_NAME = "resnet50"

LEARNING_RATE_SGD = 1e-3
LEARNING_RATE_ADAM = 1e-4
MIN_LEARNING_RATE = 1e-6
WEIGHT_DECAY = 1e-4
MOMENTUM_SGD = 0.9
BETAS_ADAM = (0.9, 0.999)
BATCH_SIZE = 16
NUM_WORKERS = 4
EPOCH_NUM = 100
SCHEDULER_PATIENCE = 5
SCHEDULER_GAMMA = 0.5
CHECKPOINTS_DIR = "checkpoints"
TENSORBOARD_DIR = "tensorboard"
RM_CHECKPOINTS_DIR = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=True,
    drop_last=True,
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=False,
    drop_last=False,
)

accelerator = Accelerator(cpu=DEVICE=="cpu", mixed_precision="no")

# model = UNet(in_channels=3, out_channels=CLASSES_NUM, bilinear=True)
model = CustomUNet(backbone_name=BACKBONE_NAME, classes_num=CLASSES_NUM)
# model = deeplabv3plus_resnet50()

# model = load_checkpoint(
#     model=model,
#     load_path=pjoin(CHECKPOINTS_DIR, "model_checkpoint_best.pt"),
# )

class_weights = torch.tensor(CLASS_WEIGHTS).to(DEVICE)
# loss_fn = nn.CrossEntropyLoss(weight=class_weights, reduction="mean", ignore_index=255)
loss_fn = CrossEntropyDiceLoss(weight=class_weights, ignore_index=255)

metric_fn = MeanIoU(classes_num=CLASSES_NUM, ignore_index=255).to(DEVICE)
metric_fns = {metric_fn.__class__.__name__: metric_fn}

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=LEARNING_RATE_SGD,
    weight_decay=WEIGHT_DECAY,
    nesterov=True,
    momentum=MOMENTUM_SGD,
)
# optimizer = torch.optim.AdamW(
#     model.parameters(), lr=LEARNING_RATE_ADAM, weight_decay=WEIGHT_DECAY, betas=BETAS_ADAM
# )

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer,
    mode="max",
    factor=SCHEDULER_GAMMA,
    patience=SCHEDULER_PATIENCE,
    min_lr=MIN_LEARNING_RATE,
)

os.makedirs(CHECKPOINTS_DIR, exist_ok=True)
checkpointer = CheckpointSaver(
    accelerator=accelerator,
    model=model,
    metric_name=metric_fn.__class__.__name__,
    save_dir=CHECKPOINTS_DIR,
    rm_save_dir=RM_CHECKPOINTS_DIR,
    max_history=5,
    should_minimize=False,
)

In [ ]:
os.makedirs(TENSORBOARD_DIR, exist_ok=True)
tensorboard_logger = torch.utils.tensorboard.SummaryWriter(log_dir=TENSORBOARD_DIR)

###################################################
# Раскомментировать в Google Colab
# %load_ext tensorboard
# %tensorboard --logdir "tensorboard"  --port 6006
###################################################

In [ ]:
model, optimizer, train_dataloader, val_dataloader, lr_scheduler = accelerator.prepare(
    model, optimizer, train_dataloader, val_dataloader, lr_scheduler
)

In [ ]:
train(
    model=model,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    loss_fn=loss_fn,
    metric_fns=metric_fns,
    lr_scheduler=lr_scheduler,
    accelerator=accelerator,
    epoch_num=EPOCH_NUM,
    checkpointer=checkpointer,
    tb_logger=tensorboard_logger,
    save_on_val=True,
)

## Загрузим и протестируем обученную модель

In [ ]:
CHECKPOINTS_DIR = "checkpoints"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# model = UNet(in_channels=3, out_channels=CLASSES_NUM)
model = CustomUNet(backbone_name=BACKBONE_NAME, classes_num=CLASSES_NUM)
# model = deeplabv3plus_resnet50()

model = load_checkpoint(
    model=model,
    load_path=pjoin(CHECKPOINTS_DIR, "model_checkpoint_best.pt"),
)
model = model.to(DEVICE)
model.eval()

In [ ]:
sample_idx = 10
image, target = val_dataset[sample_idx]

preds = torch.argmax(model(image.unsqueeze(0).to(DEVICE)).squeeze(0), axis=0)

image = image * STD[:, None, None] + MEAN[:, None, None]

fig, ax = plt.subplots(1, 3, figsize=(9, 18))
ax[0].imshow(image.numpy().transpose(1, 2, 0))
ax[1].imshow(convert_label_to_color(target.numpy()))
ax[2].imshow(convert_label_to_color(preds.cpu().numpy()));

## Разметка данных с помощью CVAT

Сайт: https://www.cvat.ai/

## Обзоры бекбонов

- Обзор до ~2020: https://arxiv.org/pdf/2206.08016.pdf
- Чуть поновее: https://arxiv.org/pdf/2310.19909.pdf
- Трансформеры и VLM позже